In [1]:
import pyspark.sql.functions as F
from core.connection import get_from_database, get_session

spark = get_session()

24/06/13 14:39:03 WARN Utils: Your hostname, IdeaPad-Gaming-3-15IHU6 resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp0s20f3)
24/06/13 14:39:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/13 14:39:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Import data from database to spark-warehouse

In [2]:
dw_ame = get_from_database(spark, "information_schema.tables")
tables = (
    dw_ame.select("table_name")
    .where(F.col("table_schema") == "public")
    .rdd.flatMap(lambda x: x)
    .collect()
)
for table in tables:
    get_from_database(spark, table).write.saveAsTable(table)

spark.sql("SHOW TABLES;").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|communications_tools|      false|
|  default|             company|      false|
|  default|             country|      false|
|  default|    operation_system|      false|
|  default|programming_language|      false|
|  default|resp_programming_...|      false|
|  default|          resp_tools|      false|
|  default|          respondent|      false|
+---------+--------------------+-----------+



1. Qual a quantidade de respondentes de cada país?

In [7]:
spark.sql(
    """
SELECT 
    c.name AS country, COUNT(c.name) AS total 
FROM 
    respondent r 
INNER JOIN 
    country c ON r.country_id = c.id 
GROUP BY country 
ORDER BY total DESC;
"""
    ).show()

+--------------------+-----+
|             country|total|
+--------------------+-----+
|       United States| 2350|
|               India| 1124|
|      United Kingdom|  749|
|             Germany|  655|
|              Canada|  360|
|              France|  278|
|  Russian Federation|  270|
|              Brazil|  255|
|              Poland|  233|
|               Spain|  203|
|           Australia|  194|
|         Netherlands|  193|
|               Italy|  166|
|              Sweden|  129|
|             Ukraine|  109|
|         Switzerland|  107|
|              Israel|  101|
|              Turkey|   98|
|               China|   92|
|Iran, Islamic Rep...|   89|
+--------------------+-----+
only showing top 20 rows

